In [22]:
import numpy as np
import scipy.stats as st
import scipy.sparse as spa
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import numpy as np
from numpy.linalg import inv

In [23]:
# Discretizing time and space

T = 2
dt = 0.001
t_grid = np.arange(0, T+dt, dt)

prodmax = 6 # maximum production
consmax = 6 # maximum consumption
h = 0.1
x_grid = np.arange(-prodmax, consmax+h, h)

In [25]:
v_true = np.zeros((len(t_grid), len(x_grid)))

for n, t in enumerate(t_grid):
    v_true[n,:] = np.sqrt(t+.5)*np.cos(x_grid)
        
# Create 3D surface plot
T, X = np.meshgrid(t_grid, x_grid)
fig = go.Figure(data=[go.Surface(x=X, y=T, z=v_true.T)])

# Customize plot
fig.update_traces(colorscale='Viridis')
fig.update_layout(scene=dict(xaxis_title='x', yaxis_title='t', zaxis_title='v(t,x)'))

# Show plot
fig.show(renderer="browser")

In [28]:
# Convection Diffusion equation - Implicit, forwards in time

v_fdm_impl = np.zeros((len(t_grid), len(x_grid)))
v_fdm_impl[:, 0] = v_true[:, 0]
v_fdm_impl[:, -1] = v_true[:, -1]
v_fdm_impl[0,:] = v_true[0, :] # Initial Condition

mu = 2 # diffusion rate
eu = 1 # transport rate

# Create matrix
up_diag = np.ones(len(x_grid[1:-2]))
diag = -2*np.ones(len(x_grid[1:-1]))
low_diag = np.ones(len(x_grid[2:-1]))
A = np.diag(up_diag, k=1) + np.diag(diag, k=0) + np.diag(low_diag, k=-1)
C = np.diag(up_diag, k=1) - np.diag(low_diag, k=-1)
M = np.eye(len(x_grid[1:-1]))-dt*mu/h**2*A+dt*eu/(2*h)*C
#print(np.linalg.cond(M))

for l, t in enumerate(t_grid[:-1]):
    # Compute and update forcing term
    f = np.array([np.cos(x)/(2*np.sqrt(t_grid[l+1]+.5))+mu*np.sqrt(t_grid[l+1]+.5)*np.cos(x)-eu*np.sqrt(t_grid[l+1]+.5)*np.sin(x) for x in x_grid[1:-1]])
    f[0] = f[0] + (mu/h**2+eu/(2*h))*v_fdm_impl[l+1, 0]
    f[-1] = f[-1] + (mu/h**2-eu/(2*h))*v_fdm_impl[l+1, -1]
    
    # Compute next layer
    v_fdm_impl[l+1, 1:-1] = np.linalg.solve(M, v_fdm_impl[l, 1:-1]+dt*f)

# Create 3D surface plot
T, X = np.meshgrid(t_grid, x_grid)
fig = go.Figure(data=[go.Surface(x=X, y=T, z=(v_true-v_fdm_impl).T)])

# Customize plot
fig.update_traces(colorscale='Viridis')
fig.update_layout(scene=dict(xaxis_title='x', yaxis_title='t', zaxis_title='v(t,x)'))

# Show plot
fig.show(renderer="browser")

In [29]:
# Convection Diffusion equation - Explicit, backwards in time

v_fdm_epx_back = np.zeros((len(t_grid), len(x_grid)))
v_fdm_epx_back[:, 0] = v_true[:, 0]
v_fdm_epx_back[:, -1] = v_true[:, -1]
v_fdm_epx_back[-1, :] = v_true[-1, :] # Terminal condition

mu = -2 # diffusion rate
eu = 1 # transport rate

# Create Matrix
up_diag = np.ones(len(x_grid[1:-2]))
diag = -2*np.ones(len(x_grid[1:-1]))
low_diag = np.ones(len(x_grid[2:-1]))
A = np.diag(up_diag, k=1) + np.diag(diag, k=0) + np.diag(low_diag, k=-1)
C = np.diag(up_diag, k=1) - np.diag(low_diag, k=-1)
M = np.eye(len(x_grid[1:-1]))-dt*mu/h**2*A+dt*eu/(2*h)*C
#print(np.linalg.cond(M))

for l, t in enumerate(t_grid[1:]):
    # iteration backwards in time
    l_back = len(t_grid)-1-l
    
    # Compute and update forcing term
    f = np.array([np.cos(x)/(2*np.sqrt(t_grid[l_back]+.5))+mu*np.sqrt(t_grid[l_back]+.5)*np.cos(x)-eu*np.sqrt(t_grid[l_back]+.5)*np.sin(x) for x in x_grid[1:-1]])
    f[0] = f[0] + (mu/h**2+eu/(2*h))*v_fdm_epx_back[l_back, 0]
    f[-1] = f[-1] + (mu/h**2-eu/(2*h))*v_fdm_epx_back[l_back, -1]
    
    # Compute next layer
    v_fdm_epx_back[l_back-1, 1:-1] = M@v_fdm_epx_back[l_back, 1:-1]-dt*f

# Create 3D surface plot
T, X = np.meshgrid(t_grid, x_grid)
fig = go.Figure(data=[go.Surface(x=X, y=T, z=(v_true-v_fdm_epx_back).T)])

# Customize plot
fig.update_traces(colorscale='Viridis')
fig.update_layout(scene=dict(xaxis_title='x', yaxis_title='t', zaxis_title='v(t,x)'))

# Show plot
fig.show(renderer="browser")

In [30]:
# Convection Diffusion equation - Explicit, forwards in time

# stability h<2*mu/eu, dt<h^2/(2*mu)

v_fdm_epx = np.zeros((len(t_grid), len(x_grid)))
v_fdm_epx[:, 0] = v_true[:, 0]
v_fdm_epx[:, -1] = v_true[:, -1]
v_fdm_epx[0, :] = v_true[0, :] # Initial condition

mu = 1 # diffusion rate
eu = 1 # transport rate

# Create matrix
up_diag = np.ones(len(x_grid[1:-2]))
diag = -2*np.ones(len(x_grid[1:-1]))
low_diag = np.ones(len(x_grid[2:-1]))
A = np.diag(up_diag, k=1) + np.diag(diag, k=0) + np.diag(low_diag, k=-1)
C = np.diag(up_diag, k=1) - np.diag(low_diag, k=-1)
M = np.eye(len(x_grid[1:-1]))+dt*mu/h**2*A-dt*eu/(2*h)*C
#print(np.linalg.cond(M))

for n, t in enumerate(t_grid[:-1]):
    # Compute and update forcing term
    f = np.array([np.cos(x)/(2*np.sqrt(t+.5))+mu*np.sqrt(t+.5)*np.cos(x)-eu*np.sqrt(t+.5)*np.sin(x) for x in x_grid[1:-1]])
    f[0] = f[0] + (mu/h**2+eu/(2*h))*v_fdm_epx[n, 0]
    f[-1] = f[-1] + (mu/h**2-eu/(2*h))*v_fdm_epx[n, -1]
    
    # Compute next layer
    v_fdm_epx[n+1, 1:-1] = M@v_fdm_epx[n, 1:-1]+dt*f

# Create 3D surface plot
T, X = np.meshgrid(t_grid, x_grid)
fig = go.Figure(data=[go.Surface(x=X, y=T, z=(v_true-v_fdm_epx).T)])

# Customize plot
fig.update_traces(colorscale='Viridis')
fig.update_layout(scene=dict(xaxis_title='x', yaxis_title='t', zaxis_title='v(t,x)'))

# Show plot
fig.show(renderer="browser")

In [32]:
# Convection Diffusion equation - Implicit, backwards in time

v_fdm_back_impl = np.zeros((len(t_grid), len(x_grid)))
v_fdm_back_impl[:, 0] = v_true[:, 0]
v_fdm_back_impl[:, -1] = v_true[:, -1]
v_fdm_back_impl[-1, :] = v_true[-1, :] # Terminal condition

mu = -0.05 # diffusion rate
eu = -3 # transport rate

# Create matrix
up_diag = np.ones(len(x_grid[1:-2]))
diag = -2*np.ones(len(x_grid[1:-1]))
low_diag = np.ones(len(x_grid[2:-1]))
A = np.diag(up_diag, k=1) + np.diag(diag, k=0) + np.diag(low_diag, k=-1)
C = np.diag(up_diag, k=1) - np.diag(low_diag, k=-1)
M = np.eye(len(x_grid[1:-1]))+dt*mu/h**2*A-dt*eu/(2*h)*C
#print(np.linalg.cond(M))

for n, t in enumerate(t_grid[1:]):    
    # iteration backwards in time
    n_back = len(t_grid)-1-n
            
    # Compute and update forcing term
    f = np.array([np.cos(x)/(2*np.sqrt(t_grid[n_back-1]+.5))+mu*np.sqrt(t_grid[n_back-1]+.5)*np.cos(x)-eu*np.sqrt(t_grid[n_back-1]+.5)*np.sin(x) for x in x_grid[1:-1]])
    f[0] = f[0] + (mu/h**2+eu/(2*h))*v_fdm_back_impl[n_back-1, 0]
    f[-1] = f[-1] + (mu/h**2-eu/(2*h))*v_fdm_back_impl[n_back-1, -1]
    
    # Compute next layer
    v_fdm_back_impl[n_back-1, 1:-1] = np.linalg.solve(M, (v_fdm_back_impl[n_back, 1:-1]-dt*f))

# Create 3D surface plot
T, X = np.meshgrid(t_grid, x_grid)
fig = go.Figure(data=[go.Surface(x=X, y=T, z=(v_true-v_fdm_back_impl).T)])

# Customize plot
fig.update_traces(colorscale='Viridis')
fig.update_layout(scene=dict(xaxis_title='x', yaxis_title='t', zaxis_title='v(t,x)'))

# Show plot
fig.show(renderer="browser")